In [20]:
import serial
import time
import os
from PIL import Image
from io import BytesIO

# Nastavení sériového portu pro ESP32-S3
usbPort = '/dev/tty.usbmodem2101'  # Mac/Linux
# usbPort = 'COM4'  # Windows
baudRate = 115200

try:
    ser = serial.Serial(usbPort, baudRate, timeout=5)
    print(f"Připojeno k {usbPort}")
except serial.SerialException:
    print(f"Chyba: Nelze otevřít {usbPort}. Zkontroluj připojení ESP32-S3!")
    exit()

# Složka pro ukládání fotek
save_dir = "photo"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Najdeme nejvyšší existující číslo
existing_images = [f for f in os.listdir(save_dir) if f.startswith('IMG_') and f.endswith('.jpg')]
if existing_images:
    last_numbers = [int(f[4:8]) for f in existing_images]
    img_counter = max(last_numbers) + 1
else:
    img_counter = 1

print(f"Další snímek bude uložen jako IMG_{img_counter:04d}.jpg")

# Hlavní smyčka
while True:
    input("Stiskni OK pro vyfocení dalšího snímku... (nebo Ctrl+C pro ukončení)")

    # Odeslání příkazu SNAP
    ser.write(b'snap\n')
    print("Posílám příkaz 'snap'...")

    buffer = b""
    capturing = False

    while True:
        line = ser.readline()
        if b"BEGIN_IMAGE" in line:
            print("Začínám přijímat snímek...")
            capturing = True
            buffer = b""
        elif b"END_IMAGE" in line:
            print("Konec snímku, zpracovávám...")

            # Převést binární buffer na obrázek
            image = Image.open(BytesIO(buffer))
            width, height = image.size
            print(f"Rozměry původního obrázku: {width}x{height}")

            # Výpočet středu a velikosti čtverce
            left = (width - height) // 2
            top = 0
            right = left + height
            bottom = height

            # Oříznutí na čtverec
            square_image = image.crop((left, top, right, bottom))
            print(f"Ořezáno na čtverec: {square_image.size}")

            filename = os.path.join(save_dir, f"IMG_{img_counter:04d}.jpg")
            square_image.save(filename, format='JPEG')
            print(f"Uloženo jako {filename}")

            img_counter += 1
            capturing = False
            break  # ukončí vnitřní while
        elif capturing:
            buffer += line

ser.close()

Připojeno k /dev/tty.usbmodem2101
Další snímek bude uložen jako IMG_0010.jpg
Posílám příkaz 'snap'...
Začínám přijímat snímek...
Konec snímku, zpracovávám...
Rozměry původního obrázku: 640x480
Ořezáno na čtverec: (480, 480)
Uloženo jako photo/IMG_0010.jpg
Posílám příkaz 'snap'...
Začínám přijímat snímek...
Konec snímku, zpracovávám...
Rozměry původního obrázku: 640x480
Ořezáno na čtverec: (480, 480)
Uloženo jako photo/IMG_0011.jpg


KeyboardInterrupt: Interrupted by user